In [13]:
# Importing all needed libraries
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [14]:
# Configurar el driver de Selenium
driver = webdriver.Chrome()
WAIT_TIME = 10
driver.wait = WebDriverWait(driver, WAIT_TIME)

In [15]:
# Función para scrapear los datos de una página
def scrape_page():
    names = []
    prices = []
    ratings = []
    
    try:
        # Encontrar todos los elementos <li> con productos
        items = driver.find_elements(By.CSS_SELECTOR, 'li.ui-search-layout__item.shops__layout-item')

        # Iterar sobre cada producto encontrado
        for item in items:
            # Verificar si existe un pie de página con la clase 'poly-footer' y dentro de él, un elemento con la clase 'poly-component__ads-promotions'
            try:
                promotion_element = item.find_element(By.CSS_SELECTOR, '.poly-footer .poly-component__ads-promotions')
                continue  # Saltamos este producto si es promocionado
            except NoSuchElementException:
                pass  # No es un producto promocionado

            # Extraer el nombre del producto
            try:
                name_element = item.find_element(By.CSS_SELECTOR, '.poly-box.poly-component__title')
                names.append(name_element.text)
            except NoSuchElementException:
                names.append('N/A')  # Si no encuentra el nombre, se coloca 'N/A'

            # Extraer el precio del producto
            try:
                price_element = item.find_element(By.CSS_SELECTOR, '.andes-money-amount__fraction')
                prices.append(price_element.text)
            except NoSuchElementException:
                prices.append('N/A')  # Si no encuentra el precio, se coloca 'N/A'

            # Extraer la calificación del producto
            try:
                rating_element = item.find_element(By.CSS_SELECTOR, '.poly-reviews__rating')
                ratings.append(rating_element.text)
            except NoSuchElementException:
                ratings.append('N/A')  # Si no encuentra la calificación, se coloca 'N/A'

    except NoSuchElementException:
        print("No se encontraron productos en la página.")
    
    return names, prices, ratings

In [16]:
def go_to_next_page():
    try:
        # Espera explícita para asegurarte de que el botón es clicable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.andes-pagination__button--next a'))
        )
        # Hacer clic en el botón
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(5)  # Esperar un poco para que la página siguiente cargue
    except (NoSuchElementException, TimeoutException):
        print("Botón 'Siguiente' no encontrado o no interactuable. Fin del scraping.")
        return False
    return True

In [17]:
# Abrir el navegador y cargar la URL
url = "https://listado.mercadolibre.com.ar/smartphone#D[A:smartphone]"
driver.get(url)
time.sleep(5)  # Esperar a que la página cargue


In [18]:
# Listas para almacenar los datos de todas las páginas
all_names = []
all_prices = []
all_ratings = []

In [19]:
# Ciclo para procesar las primeras 6 páginas
for page in range(3):
    print(f"Scrapeando página {page + 1}")
    
    # Extraer datos de la página actual
    names, prices, ratings = scrape_page()
    all_names.extend(names)
    all_prices.extend(prices)
    all_ratings.extend(ratings)
    
    # Ir a la siguiente página
    if not go_to_next_page():
        break

Scrapeando página 1
Scrapeando página 2
Scrapeando página 3


In [20]:
# Crear el DataFrame con los datos extraídos
df = pd.DataFrame({
    'Name': all_names,
    'Price': all_prices,
    'Rating': all_ratings
})

In [21]:
# Mostrar el DataFrame
print(df)


                                                  Name    Price Rating
0                  TCL 40 SE 128 GB dark gray 4 GB RAM  299.999    4.6
1    Tecno Spark 20 Pro Dual SIM 256 GB blanco 8 GB...  549.999    4.6
2    Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB neg...  849.999    4.8
3         Moto G24 Power Dual SIM 256 GB azul 8 GB RAM  273.681    4.6
4             Motorola Moto G24 4gb + 128 Gb Ice Green  370.990    4.7
..                                                 ...      ...    ...
145                    Samsung S20 Fe 128 Gb 6 Ram Nfc  485.000    N/A
146                          Smarthphone Samsung S21fe  430.000    N/A
147                     Samsung Galaxy S21 Fe 128gb 5g  800.000    N/A
148  Lote Celular Smartphone Samsung Gti S4 S3 Gran...    1.992    N/A
149                   Smartphone Blackberry Curve 9320   25.000    N/A

[150 rows x 3 columns]
